<a href="https://colab.research.google.com/github/lizaoh/smp_program_data/blob/main/smp2016_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of Script

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymupdf
!pip install pymupdf-layout
!pip install pymupdf4llm
!pip install wordfreq
# !pip install rapidfuzz
import glob
import os
import pathlib
import pymupdf
import pymupdf.layout
import pymupdf4llm
import re
import pandas as pd
import unicodedata
import wordfreq
# from rapidfuzz import process, fuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 149.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: PyMuPDF
    Found existing installation: PyMuPDF 1.26.7
    Uninstalling PyMuPDF-1.26.7:
      Successfully uninstalled PyMuPDF-1.26.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.1/183.1 kB 18.3 MB/s eta 0:00:00


In [3]:
pdfs_path = '/content/drive/MyDrive/math_psych_work/Conference Programs/'

# Functions
Created with help from GPT 5.2, but some are my own code just turned into a function.

In [4]:
# Authors are always [last name], [first name, sometimes middle initial] and
# separated by commas, with last author having ", and" before it
AUTHOR_LINE_RE = re.compile(
    r"""
    ^
    # First author
    (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,      # last name (with lowercase particles)
    \s+
    (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)            # first name OR initial
    (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*    # middle names / initials

    # Additional authors
    (?:,\s+
        (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,
        \s+
        (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)
        (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*
    )*

    # Optional ", and Last, First"
    (?:,\s+and\s+
        (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,
        \s+
        (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)
        (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*
    )?

    \.?
    $
    """,
    re.VERBOSE
)

def is_author_only_line(line: str) -> bool:
    return bool(AUTHOR_LINE_RE.match(line.strip()))

In [5]:
def parse_authors(line: str) -> list[str]:
    line = line.strip()
    line = re.sub(r',?\s*and\s*$', '', line)
    line = line.replace(', and ', ', ')

    parts = [p.strip() for p in line.split(',') if p.strip()]

    if len(parts) % 2 != 0:
        # optional: log or inspect these cases
        parts = parts[:-1]

    return [
        f"{parts[i+1]} {parts[i]}"
        for i in range(0, len(parts), 2)
    ]

In [6]:
def split_authors_and_abstract_fallback(text: str):
    """
    Handles cases where authors and abstract are on the same line.
    """
    for match in AUTHOR_LINE_RE.finditer(text):
        end = match.end()
        authors = text[:end].strip()
        abstract = text[end:].strip()
        return authors, abstract
    return None, None

In [7]:
def rehyphenate_words(text, words_to_hyphenate):
    for word, hyphenated_word in words_to_hyphenate:
        text = text.replace(word, hyphenated_word)

    return text

In [8]:
def remove_trailing_text(text):
  no_trailing_junk = text.split(".")[:-1]

  return ".".join(no_trailing_junk) + "."

In [9]:
def clean_text(text, words_to_hyphenate=None):
    if not text:
        return text

    if words_to_hyphenate:
      text = rehyphenate_words(text, words_to_hyphenate)

    text = re.sub(r'`"`\s(.*?)\s`"`', r'"\1"', text)  # Replace `"` with "
                                                      # and getting rid of extra
                                                      # space between the quotes
                                                      # and the first and last words

    text = re.sub(r' \n\n\d{1,3} \n\n', ' ', text)  # Remove page breaks with page number
    text = re.sub(r'\n\n\d{2}', '', text)

    text = re.sub(r'-\s+(?!\b(?:and|or)\b)', '', text)  # Get rid of - and space after
                                                        # unless word after is
                                                        # "and" or "or"

    text = text.strip()
    text = fix_ligatures(text)

    return text

In [10]:
LIGATURE_MAP = {
    "ﬁ": "fi", "ﬂ": "fl", "ﬃ": "ffi", "ﬄ": "ffl", "ﬀ": "ff", "ﬅ": "ft", "ﬆ": "st",
    "Æ": "ffi", "¨u": "ü", "¨a": "ä", "´e": "é", "`e": "è", "`a": "à", "¨o": "ö",
    "˚a": "å", "c¸": "ç", '“': '"', '”': '"', "’": "'", '˜n': 'ñ', 'ˇs': 'š',
    "âĂŸ": "'", "``": '"'
}

def fix_ligatures(text):
    # Replace known ligatures
    for bad, good in LIGATURE_MAP.items():
        text = text.replace(bad, good)

    # Replace any private-use ligature (common in PDFs)
    cleaned_chars = []
    for ch in text:
        name = unicodedata.name(ch, "")
        if "LIGATURE" in name.upper():
            # Try to break it apart: remove spaces and lowercase
            base = name.split("LIGATURE")[-1]
            base = base.replace(" ", "").lower()
            cleaned_chars.append(base)
        else:
            cleaned_chars.append(ch)

    return "".join(cleaned_chars)

In [11]:
# Checks if valid word using Zipf frequency
def is_probably_valid(word, threshold=2.5):
    return wordfreq.zipf_frequency(word, "en") > threshold  # smaller number cuts off
                                                            # more words, bigger is
                                                            # more lenient

# Program

138 entries total (4 plenary talks, 24 symposium talks, 88 talks, and 22 posters)

Markdown shows bold and italic text here, but the extracted text doesn't show the authors. Author names are "last name, first name." Also no affiliations listed.


## Grab text from the pdf

In [12]:
year = '2016'
program = pymupdf.open(pdfs_path + f'smp{year}_program.pdf')

In [13]:
program_text = pymupdf4llm.to_markdown(program)

=== Document parser messages ===
Full-page OCR on page.number=0/1.



In [23]:
program_text[6_400:8_000]

's section is automatically generated from the submission website and may not reflect recent updates, such as withdrawn contributions or changes in authorship. \n\nFirst appear the keynote addresses, then the symposium presentations, then the regular submitted presentations, and finally the posters. \n\n## Keynote Addresses \n\n## Digging Deeper into the Temporal Structure of the Mind \n\nAnderson, John R. \n\nNeural imaging can be used to identify the stages of mental processes.   We have used a combination of hidden semi-Markov models and multivariate pattern matching (HSMM-MVPA) to analyze MEG data from memory experiments.   The HSMM-MVPA methods are able to reveal brief bumps in the sensor data that mark the onset of different stages of processing. We used these bumps to isolate encoding, retrieval, decision, and response stages within single memory trials. The temporal distributions of these stages as well as the brain regions engaged indicate that the same stages occur in two rat

## Split text into presentation entries

In [24]:
all_abstracts = program_text.split('\n\n## Keynote Addresses ')[1] # this is where abstracts start
split_abstracts = re.split(r'\n\n##', all_abstracts)
abstract_entries = [entry.strip() for entry in split_abstracts if entry.strip()][:-1]

In [25]:
abstract_entries[:2]

['Digging Deeper into the Temporal Structure of the Mind \n\nAnderson, John R. \n\nNeural imaging can be used to identify the stages of mental processes.   We have used a combination of hidden semi-Markov models and multivariate pattern matching (HSMM-MVPA) to analyze MEG data from memory experiments.   The HSMM-MVPA methods are able to reveal brief bumps in the sensor data that mark the onset of different stages of processing. We used these bumps to isolate encoding, retrieval, decision, and response stages within single memory trials. The temporal distributions of these stages as well as the brain regions engaged indicate that the same stages occur in two rather different experiments (associative recognition of word pairs versus recall of arithmetic facts).  This new technique allows the retrieval stage to be isolated from the rest of a trial. This in turn allows us to test theoretical claims concerning the distribution of retrieval times and how they vary with condition.',
 'Informi

## Find words to re-hyphenate

In [26]:
pattern = re.compile(r'([A-Za-z]+)-\n\s*([A-Za-z]+)')
possible_hyphenated_words = []

counter = 0
for p, page in enumerate(program[6:]):  # these are the pages with abstracts only
  text = fix_ligatures(page.get_text('text'))
  matches = pattern.findall(text)

  for left, right in matches:
    word = f"{left}{right}"
    hyphenated_word = f"{left}-{right}"
    if not is_probably_valid(word, threshold=2.8):
      possible_hyphenated_words.append([word, hyphenated_word])

      print(f"{counter:>3}: Page {p+7:<3} {hyphenated_word:<30} {word}")
      counter += 1

  0: Page 7   non-interacting                noninteracting
  1: Page 10  long-term                      longterm
  2: Page 12  high-contrast                  highcontrast
  3: Page 14  mind-wandering                 mindwandering
  4: Page 19  long-range                     longrange
  5: Page 21  well-recovered                 wellrecovered
  6: Page 21  spatially-driven               spatiallydriven
  7: Page 26  third-person                   thirdperson
  8: Page 28  by-Default                     byDefault
  9: Page 33  Long-Term                      LongTerm
 10: Page 34  double-detection               doubledetection
 11: Page 34  rate-distortion                ratedistortion
 12: Page 36  single-region                  singleregion
 13: Page 37  Multi-Attribute                MultiAttribute
 14: Page 44  eye-movements                  eyemovements
 15: Page 46  pre-programmed                 preprogrammed
 16: Page 46  socio-cognitive                sociocognitive
 17: Page 48

In [27]:
# Pick indices of words to rehyphenate
indices = [(0, 14), 17, (19, 28)]
words_to_hyphenate = [
    possible_hyphenated_words[i]
    for item in indices
    for i in ([item] if isinstance(item, int) else range(*item))
]

In [28]:
for word, hyphenated_word in words_to_hyphenate:
  print(f'{word:<30} {hyphenated_word}')

noninteracting                 non-interacting
longterm                       long-term
highcontrast                   high-contrast
mindwandering                  mind-wandering
longrange                      long-range
wellrecovered                  well-recovered
spatiallydriven                spatially-driven
thirdperson                    third-person
byDefault                      by-Default
LongTerm                       Long-Term
doubledetection                double-detection
ratedistortion                 rate-distortion
singleregion                   single-region
MultiAttribute                 Multi-Attribute
wellknown                      well-known
realvalued                     real-valued
hillclimbing                   hill-climbing
Gargtype                       Garg-type
wellknown                      well-known
DecisionMaking                 Decision-Making
bymoment                       by-moment
discretestate                  discrete-state
receiveroperating       

## Sort authors, affiliations, title, and abstract

In [29]:
parsed_entries = []
temp_title = None

for entry in abstract_entries:
  number_of_splits = 1 if temp_title else 2
  cleaned_entry = clean_text(entry, words_to_hyphenate)
  entry_parts = cleaned_entry.split('\n\n', number_of_splits)

  # If no split happened, maybe it's a title separated from authors and abstract
  if len(entry_parts) == 1:
    temp_title = cleaned_entry
    continue

  # If only 2 parts, maybe previous element was the title and this is just authors
  # and abstract
  if len(entry_parts) == 2:
    if temp_title is None:
        # Maybe no new lines between authors and abstract
        authors, abstract = split_authors_and_abstract_fallback(entry_parts[1])
        if authors is None:
            continue
        title = entry_parts[0].strip()
    else:
        title = temp_title
        temp_title = None
        authors, abstract = entry_parts

  # Otherwise sort title, authors, abstract (no affiliations in this program)
  elif len(entry_parts) == 3:
    title, authors, abstract = entry_parts

  cleaned_authors = parse_authors(authors.strip())
  cleaned_authors = ', '.join(cleaned_authors)

  cleaned_abstract = re.sub(r'\s+', ' ', abstract.strip())
  cleaned_abstract = remove_trailing_text(cleaned_abstract)

  parsed_entries.append({
    'year': year,
    'author(s)': cleaned_authors,
    'affiliation(s)': '',
    'title': title.strip(),
    'type': '',
    'abstract': cleaned_abstract
  })

In [35]:
parsed_entries[:2]

[{'year': '2016',
  'author(s)': 'John R. Anderson',
  'affiliation(s)': '',
  'title': 'Digging Deeper into the Temporal Structure of the Mind',
  'type': '',
  'abstract': 'Neural imaging can be used to identify the stages of mental processes. We have used a combination of hidden semi-Markov models and multivariate pattern matching (HSMM-MVPA) to analyze MEG data from memory experiments. The HSMM-MVPA methods are able to reveal brief bumps in the sensor data that mark the onset of different stages of processing. We used these bumps to isolate encoding, retrieval, decision, and response stages within single memory trials. The temporal distributions of these stages as well as the brain regions engaged indicate that the same stages occur in two rather different experiments (associative recognition of word pairs versus recall of arithmetic facts). This new technique allows the retrieval stage to be isolated from the rest of a trial. This in turn allows us to test theoretical claims conce

# Create df and convert to csv

In [31]:
df = pd.DataFrame(parsed_entries, columns=["year", "author(s)", "affiliation(s)", "title", "type", "abstract"])

In [32]:
df.head()

,year,author(s),affiliation(s),title,type,abstract
0,2016,John R. Anderson,,Digging Deeper into the Temporal Structure of ...,,Neural imaging can be used to identify the sta...
1,2016,Brandon (William K. Estes Early Career Award R...,,Informing Cognitive Abstractions with Neurophy...,,Our understanding of cognition has been advanc...
2,2016,Chris (William K. Estes Early Career Award Rec...,,"A Less Simple, but More Complete Model of Choi...",,Brown & Heathcote's (2008) Linear Ballistic Ac...
3,2016,,,**Symposium:** _**Integrating Prior Knowledge ...,,"Jacobs, Robert Visual working memory (VWM) hel..."
4,2016,Sam Gershman,,Memory and Structure,,"When do we modify old memories, and when do we..."


In [33]:
df.to_csv(f"/content/drive/MyDrive/math_psych_work/csv/smp{year}_program.csv", index=False)